# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

**Импортируем все необходимые библиотеки**

In [44]:
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

**Рассмотрим основные показатели датасета**

In [45]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [47]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Посмотрим как распределены классы в колонке is_ultra в нашей выборке.**

In [48]:
display(df['is_ultra'].value_counts())
f'Доля значений "True" в выборке: {df["is_ultra"].mean()}'

0    2229
1     985
Name: is_ultra, dtype: int64

'Доля значений "True" в выборке: 0.30647168637212197'

**Рассчитаем коэффицент корреляции Пирсона для пар параметров будущей модели**

In [49]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']
for column_1 in range(len(features)):
    slice_f = column_1 + 1
    for column_2 in features.columns[slice_f:]:
        first_col = features.iloc[:, column_1]
        second_col = features[column_2]
        correlation = first_col.corr(second_col)
        print(f'Корреляция между колонками {first_col.name} и {second_col.name} равна {correlation}')

Корреляция между колонками calls и minutes равна 0.9820832355742293
Корреляция между колонками calls и messages равна 0.1773845012176953
Корреляция между колонками calls и mb_used равна 0.28644151203505447
Корреляция между колонками minutes и messages равна 0.1731102258096908
Корреляция между колонками minutes и mb_used равна 0.2809669350882149
Корреляция между колонками messages и mb_used равна 0.19572112095582148


**В результате изучения основных характеристик данных можно сделать следующие выводы. В данных отсутствуют пропуски. Каждая колонка содержит 3214 значений. Поведение пользователя характеризуется следующими колонками: calls, minutes, messages, mb_used, is_ultra. 1 строка соответствует данным об 1 пользователе. Признаки для обучения будут храниться во всех колонках кроме целевой - "is_ultra". "is_ultra" содержит в себе бинарные категориальные данные, соответственно перед нами стоит задача бинарной классификации с помощью обучения с учителем. В данных также наблюдается определённый дисбаланс классов. Значений True в выборке порядка 30%. Это необходимо будет учесть при разделении данных и последующем построении моделей. Существенная корреляция наблюдается только между колонками calls и minutes.**

## Разбейте данные на выборки

**Разделим данные на выборки и проверим корректность разделения выводом на экран информации о размерности получившихся выборок.**

In [50]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)
sample_dict = {'features_train': features_train,
               'features_valid': features_valid,
               'features_test': features_test,
               'target_train': target_train,
               'target_valid': target_valid,
               'target_test': target_test}
for name, sample in sample_dict.items():
    print(f'Количество строк в выборке {name}: {sample.shape[0]}, \
количество столбцов: {sample.shape[1] if len(sample.shape) > 1 else 1}')

Количество строк в выборке features_train: 1928, количество столбцов: 4
Количество строк в выборке features_valid: 643, количество столбцов: 4
Количество строк в выборке features_test: 643, количество столбцов: 4
Количество строк в выборке target_train: 1928, количество столбцов: 1
Количество строк в выборке target_valid: 643, количество столбцов: 1
Количество строк в выборке target_test: 643, количество столбцов: 1


**В данных есть числовые признаки (причём резко отличаюшиеся по значению модуля: есть значения около 1 (маленькие, например messages) и десятки тысяч (большие, например mb_used). Такой разброс может резко негативно сказаться на точности прогнозов линейных моделей. Проведём стандартизацию данных.**

In [51]:
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_valid_scaled = scaler.transform(features_valid)
features_test_scaled = scaler.transform(features_test)

**Мы разбили данные на 3 подгруппы: обучающие, валидационные и тестовые в процентном соотнощении 60%:20%:20% соответственно.**

## Исследуйте модели

**Проверим три модели: DecisionTreeClassifier, RandomForestClassifier, LogisticRegression. Обучим модели на обучающей выборке с различными гиперпараметрами, выберим 3 модели с гиперпараметрами, дающими лучший результат. Сравним данные accuracy по каждой из моделей на валидационной выборке и ранжируем их по эффективности.**

### Решающее дерево

In [52]:
min_leaf, min_split, best_depth, max_accuracy = 0, 0, 0, 0
best_model_tree = None
for i in range(1, 6):
    for j in range(1, 10):
        for k in range(2, 10):
            model_tree = DecisionTreeClassifier(max_depth=i, random_state=12345, min_samples_leaf=j, min_samples_split=k)
            model_tree.fit(features_train_scaled, target_train)
            accuracy = accuracy_score(target_valid, model_tree.predict(features_valid_scaled))
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                best_depth = i
                best_model_tree = model_tree
                min_split = k
                min_leaf = j
print(f'Максимальное качество прогноза:{max_accuracy}. \
Достигнуто при показателе max_depth = {best_depth}, \
min_samples_leaf = {min_leaf}, min_samples_split = {min_split}')

Максимальное качество прогноза:0.7853810264385692. Достигнуто при показателе max_depth = 5 2, min_samples_leaf = 1, min_samples_split = 2


### Случайный лес

In [53]:
best_model_forest = None
best_result = 0
best_estimator = 0
best_depth = 0
best_leaf = 0
for est in range(1, 11):
    for depth in range (1, 11):
        for leaf in range(1, 10):
            model_forest = RandomForestClassifier(
                random_state=12345, n_estimators=est,
                max_depth=depth,
                min_samples_leaf=leaf)
            model_forest.fit(features_train_scaled, target_train)
            accuracy = accuracy_score(target_valid, model_forest.predict(features_valid_scaled))
            if accuracy > best_result:
                best_model_forest = model_forest
                best_result = result
                best_estimator = est
                best_depth = depth
                best_leaf = leaf

print(f'Максимальное качество прогноза:{best_result}. \
Достигнуто при показателе n_estimators = {best_estimator},  max_depth = {best_depth}, min_samples_leaf = {best_leaf}')

Максимальное качество прогноза:0.8087091757387247. Достигнуто при показателе n_estimators = 10,  max_depth = 9, min_samples_leaf = 9


### Логистическая регрессия

In [54]:
best_solver = None
is_fit_intercept = 0
best_model_logistic = None
max_logistic_accuracy = 0
for solv in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']:
    for inter in range(2):
        model_logistic = LogisticRegression(solver=solv, max_iter=1000, random_state=12345, fit_intercept=inter)
        model_logistic.fit(features_train_scaled, target_train)
        logistic_accuracy = accuracy_score(target_valid, model_logistic.predict(features_valid_scaled))
        if logistic_accuracy > max_logistic_accuracy:
            max_logistic_accuracy = logistic_accuracy
            best_model_logistic = model_logistic
            best_solver = solv
            is_fit_intercept = inter
print(f'Максимальное качество прогноза:{max_logistic_accuracy}. \
Достигнуто при показателе fit_intercept = {is_fit_intercept}, solver = {best_solver}')

Максимальное качество прогноза:0.7387247278382582. Достигнуто при показателе fit_intercept = 1, solver = liblinear


**Исходя из приведённых выше рассчётов, по качеству прогнозов на валидационной выборке модели ранжируются следующим образом:**
1. Случайный лес
2. Решающее дерево
3. Логистическая регрессия

<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Здесь отлично: подобрали для всех наших моделей лучшие гиперпараметры (в данном случае - максимизирующие метрику accuracy_score). Также здесь мы ещё и определили САМУЮ лучшую модель. На валидации ею оказалась модель "случайного леса". 

После того, как гиперпараметры на валидации подобраны - тестируем модели на тестовых данных. По результатам тестирования на тесте (сорри за тавталогию) выбираем модель, которую сможем передать в продакшн. 
</div>

## Проверьте модель на тестовой выборке

**Лучшие результаты на валидационной выборке показала модель Случайного леса. Проверим результаты этой модели на тестовой выборке.**

In [55]:
test_accuracy = accuracy_score(target_test, best_model_forest.predict(features_test_scaled))
print(f'Качество прогноза на тестовых данных у модели Случайный лес: {test_accuracy}')    

Качество прогноза на тестовых данных у модели Случайный лес: 0.8118195956454122


**Модель Случайный лес прошла проверку на тестовых данных с результатом accuracy = 0.81.**

## Проверьте модели на адекватность

**Проверим модель на адекватность сравнив с результатом accuracy_score, которые даёт применение на тестовых данных модели DummyClassifier. Так как данные в целевом признаке неравномерны, будем использовать DummyClassifier с параметром strategy='most_frequent'.**

In [56]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_model.fit(features_train_scaled, target_train)
dummy_accuracy = accuracy_score(target_test, dummy_model.predict(features_test_scaled))
dummy_diff = (test_accuracy - dummy_accuracy)
print(f'Точность модели DummyClassifier = {dummy_accuracy}. \
Разница между предсказаниями составляет {dummy_diff:.2%}')

Точность модели DummyClassifier = 0.6936236391912908. Разница между предсказаниями составляет 11.82%


**В результате проверки модели на адекватность мы выяснили, что выбранная модель показывает результаты на 11.82% лучше примитивной.**

## Общий вывод

**Для работы с моделями был подготовлен датафрейм с данными о поведении пользователей ряда тарифов мобильной связи. Данные были проанализированы, разбиты на 3 группы выборок: обучающие, валидационные и тестовые в пропорции 3:1:1 соответственно. Признаки были стандартизованы с использованием StandardScaler. В результате проведённой работы было обучено три типа моделей: Решающее дерево, Случайный лес и Логистическая регрессия. Для каждого типа моделей экспериментальным путём были определены оптимальные гиперпараметры. Перед нами стояла задача бинарной классификации по наиболее предпочтительному тарифу. Лучше всего с задачей справилась модель Случайного леса. Она предсказывает поведение пользователей с точностью 0.81 на тестовых данных Это на 11.82% выше, чем показатели примитивной модели DummyClassifier.**